# Translation with Azure AI Translator

https://learn.microsoft.com/en-us/azure/ai-services/translator/text-translation-overview

<img src="webapp1.jpg">

In [1]:
# %pip install azure-ai-translation-text==1.0.0b1

In [22]:
import datetime
import gradio as gr
import pandas as pd
import sys

from azure.ai.translation.text import TextTranslationClient, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError
from dotenv import load_dotenv

In [3]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 11-Apr-2024 12:41:02


## Azure AI Translator credentials

In [5]:
load_dotenv("azure.env")

azure_ai_translator_key = os.getenv("AZURE_AI_TRANSLATION_KEY")
azure_ai_translator_endpoint = os.getenv("AZURE_AI_TRANSLATION_ENDPOINT")
azure_ai_translator_region = os.getenv("AZURE_AI_TRANSLATION_REGION")

In [6]:
credential = TranslatorCredential(azure_ai_translator_key, azure_ai_translator_region)

text_translator = TextTranslationClient(
    endpoint=azure_ai_translator_endpoint, credential=credential)

## Testing

In [7]:
try:
    target_languages = ["fr"]
    input_text_elements = [InputTextItem(text="Azure is a cloud computing platform run by Microsoft. It offers access, management, and the development of applications and services through global data centers")]

    response = text_translator.translate(
        content=input_text_elements, to=target_languages)
    translation = response[0] if response else None

    if translation:
        detected_language = translation.detected_language
        print("\033[1;31;34m")
        if detected_language:
            print(
                f"Detected languages of the input text: {detected_language.language} with score = {detected_language.score}"
            )
        print()
        for translated_text in translation.translations:
            print(
                f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

except HttpResponseError as exception:
    if exception.error is not None:
        print(f"Error Code: {exception.error.code}")
        print()
        print(f"Message: {exception.error.message}")
    raise


Detected languages of the input text: en with score = 1.0

Text was translated to: 'fr' and the result is: 'Azure est une plateforme de cloud computing gérée par Microsoft. Il offre l’accès, la gestion et le développement d’applications et de services via des centres de données mondiaux'.


In [8]:
try:
    target_languages = ["ko"]
    input_text_elements = [InputTextItem(text="Merci encore. A bientôt.")]

    response = text_translator.translate(
        content=input_text_elements, to=target_languages)
    translation = response[0] if response else None

    if translation:
        detected_language = translation.detected_language
        print("\033[1;31;34m")
        
        if detected_language:
            print(
                f"Detected languages of the input text: {detected_language.language} with score = {detected_language.score}."
            )
            print()
        for translated_text in translation.translations:
            print(
                f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

except HttpResponseError as exception:
    if exception.error is not None:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")
    raise


Detected languages of the input text: fr with score = 1.0.

Text was translated to: 'ko' and the result is: '다시 한 번 감사드립니다. 곧  봐요.'.


In [9]:
mytext = "Quel est le temps de trajet de Paris à Versailles ?"

In [10]:
try:
    source_language = "fr"
    target_languages = ["es", "it", "de", "en",
                        "el", "mn-Mong", "ko", "fa", "ru", "ar"]
    input_text_elements = [InputTextItem(text=mytext)]

    response = text_translator.translate(content=input_text_elements,
                                         to=target_languages,
                                         from_parameter=source_language)
    translation = response[0] if response else None

    print("Text to translate:", mytext)

    if translation:
        print("\033[1;31;34m")
        for translated_text in translation.translations:
            print(
                f"Translated to code: '{translated_text.to}' ==> '{translated_text.text}'.")

except HttpResponseError as exception:
    print(f"Error Code: {exception.error.code}")
    print(f"Message: {exception.error.message}")

Text to translate: Quel est le temps de trajet de Paris à Versailles ?

Translated to code: 'es' ==> '¿Cuál es el tiempo de viaje de París a Versalles?'.
Translated to code: 'it' ==> 'Qual è la durata del viaggio da Parigi a Versailles?'.
Translated to code: 'de' ==> 'Wie lange dauert die Fahrt von Paris nach Versailles?'.
Translated to code: 'en' ==> 'What is the travel time from Paris to Versailles?'.
Translated to code: 'el' ==> 'Ποιος είναι ο χρόνος ταξιδιού από το Παρίσι στις Βερσαλλίες;'.
Translated to code: 'mn-Mong' ==> 'ᠫᠠᠷᠢᠰ ᠠᠴᠠ ᠹᠠᠨᠯᠰᠠᠢ ᠣᠷᠳᠣᠨ ᠬᠦᠷᠬᠦ ᠵᠢᠭᠤᠯᠴᠢᠯᠠᠯ ᠤᠨ ᠴᠠᠭ ᠨᠢ ᠬᠡᠳᠦᠢ ᠪᠤᠢ ?'.
Translated to code: 'ko' ==> '파리에서 베르사유까지가는 데 걸리는 시간은 얼마입니까?'.
Translated to code: 'fa' ==> 'زمان سفر از پاریس به ورسای چقدر است؟'.
Translated to code: 'ru' ==> 'Сколько времени занимает поездка из Парижа в Версаль?'.
Translated to code: 'ar' ==> 'ما هو وقت السفر من باريس إلى فرساي؟'.


## Webapp

https://learn.microsoft.com/en-us/azure/ai-services/translator/language-support

In [11]:
credential = TranslatorCredential(azure_ai_translator_key, azure_ai_translator_region)

text_translator = TextTranslationClient(
    endpoint=azure_ai_translator_endpoint, credential=credential)

In [12]:
try:
    response = text_translator.get_languages()

    print(
        f"Number of supported languages for translate operation: {len(response.translation) if response.translation is not None else 0}"
    )
    print(
        f"Number of supported languages for transliterate operation: {len(response.transliteration) if response.transliteration is not None else 0}"
    )
    print(
        f"Number of supported languages for dictionary operations: {len(response.dictionary) if response.dictionary is not None else 0}"
    )

    print()

    if response.translation is not None:
        print("Translation Languages:")
        i = 1
        for key, value in response.translation.items():
            print(f"{i} {key} -- name: {value.name} ({value.native_name})")
            i += 1

except HttpResponseError as exception:
    if exception.error is not None:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")
    raise

Number of supported languages for translate operation: 135
Number of supported languages for transliterate operation: 42
Number of supported languages for dictionary operations: 50

Translation Languages:
1 af -- name: Afrikaans (Afrikaans)
2 am -- name: Amharic (አማርኛ)
3 ar -- name: Arabic (العربية)
4 as -- name: Assamese (অসমীয়া)
5 az -- name: Azerbaijani (Azərbaycan)
6 ba -- name: Bashkir (Bashkir)
7 bg -- name: Bulgarian (Български)
8 bho -- name: Bhojpuri (Bhojpuri)
9 bn -- name: Bangla (বাংলা)
10 bo -- name: Tibetan (བོད་སྐད་)
11 brx -- name: Bodo (बड़ो)
12 bs -- name: Bosnian (Bosnian)
13 ca -- name: Catalan (Català)
14 cs -- name: Czech (Čeština)
15 cy -- name: Welsh (Cymraeg)
16 da -- name: Danish (Dansk)
17 de -- name: German (Deutsch)
18 doi -- name: Dogri (Dogri)
19 dsb -- name: Lower Sorbian (Dolnoserbšćina)
20 dv -- name: Divehi (ދިވެހިބަސް)
21 el -- name: Greek (Ελληνικά)
22 en -- name: English (English)
23 es -- name: Spanish (Español)
24 et -- name: Estonian (Eesti)
25

In [13]:
data = []

if response.translation is not None:
    for key, value in response.translation.items():
        data.append({'Language_Code': key,
                     'Language_Name': value.name,
                     'Native_Name': value.native_name})

df_languages = pd.DataFrame(data)
df_languages

,Language_Code,Language_Name,Native_Name
0,af,Afrikaans,Afrikaans
1,am,Amharic,አማርኛ
2,ar,Arabic,العربية
3,as,Assamese,অসমীয়া
4,az,Azerbaijani,Azərbaycan
...,...,...,...
130,yua,Yucatec Maya,Yucatec Maya
131,yue,Cantonese (Traditional),粵語 (繁體)
132,zh-Hans,Chinese Simplified,中文 (简体)
133,zh-Hant,Chinese Traditional,繁體中文 (繁體)


In [23]:
language_dict = df_languages.set_index('Language_Code')['Language_Name'].to_dict()
language_dict

{'af': 'Afrikaans',
 'am': 'Amharic',
 'ar': 'Arabic',
 'as': 'Assamese',
 'az': 'Azerbaijani',
 'ba': 'Bashkir',
 'bg': 'Bulgarian',
 'bho': 'Bhojpuri',
 'bn': 'Bangla',
 'bo': 'Tibetan',
 'brx': 'Bodo',
 'bs': 'Bosnian',
 'ca': 'Catalan',
 'cs': 'Czech',
 'cy': 'Welsh',
 'da': 'Danish',
 'de': 'German',
 'doi': 'Dogri',
 'dsb': 'Lower Sorbian',
 'dv': 'Divehi',
 'el': 'Greek',
 'en': 'English',
 'es': 'Spanish',
 'et': 'Estonian',
 'eu': 'Basque',
 'fa': 'Persian',
 'fi': 'Finnish',
 'fil': 'Filipino',
 'fj': 'Fijian',
 'fo': 'Faroese',
 'fr': 'French',
 'fr-CA': 'French (Canada)',
 'ga': 'Irish',
 'gl': 'Galician',
 'gom': 'Konkani',
 'gu': 'Gujarati',
 'ha': 'Hausa',
 'he': 'Hebrew',
 'hi': 'Hindi',
 'hne': 'Chhattisgarhi',
 'hr': 'Croatian',
 'hsb': 'Upper Sorbian',
 'ht': 'Haitian Creole',
 'hu': 'Hungarian',
 'hy': 'Armenian',
 'id': 'Indonesian',
 'ig': 'Igbo',
 'ikt': 'Inuinnaqtun',
 'is': 'Icelandic',
 'it': 'Italian',
 'iu': 'Inuktitut',
 'iu-Latn': 'Inuktitut (Latin)',
 'ja

In [15]:
language_full_names = list(language_dict.values())
language_full_names.sort()
len(language_full_names)

135

In [16]:
language_codes = list(language_dict.keys())
language_codes.sort()

reverse_language_names = {v: k for k, v in language_dict.items()}

In [17]:
def get_language_code(language_name):
    # Function to get language code by language name
    return reverse_language_names.get(language_name)

In [18]:
# Quick test
get_language_code("French")

'fr'

In [19]:
def translate_fn(mytext, source_lang, target_lang):
    """
    Translate the text from source lang to target lang
    """
    source_lang = get_language_code(source_lang)
    target_lang = get_language_code(target_lang)

    target_lang = [target_lang]
    input_text_elements = [InputTextItem(text=mytext)]
    response = text_translator.translate(content=input_text_elements,
                                         to=target_lang,
                                         from_parameter=source_lang)

    resp = response[0] if response else None

    if resp:
        for translated_text in resp.translations:
            result = translated_text.text

    return result

In [20]:
# Quick test
translate_fn("hello", "English", "French")

'Bonjour'

In [21]:
image_url = "https://th.bing.com/th/id/OIP.ngRD8aNJwXZ2B0eol5tFlwAAAA?rs=1&pid=ImgDetMain"
logo = "<center> <img src= {} width=100px></center>".format(image_url)

translator_webapp = gr.Interface(
    fn=translate_fn,
    inputs=[
        gr.components.Textbox(label="Text to translate"),
        gr.components.Dropdown(label="Source language",
                               choices=language_full_names),
        gr.components.Dropdown(label="Target language",
                               choices=language_full_names),
    ],
    outputs=gr.Text(label="Translated text"),
    cache_examples=False,
    title="Azure AI Translator",
    description=logo,
    theme="JohnSmith9982/small_and_pretty",
    examples=[
        [
            "Hello. Welcome to this presentation of Azure AI Translator",
            "English",
            "French",
        ],
        [
            "Hola. Bienvenidos a esta presentación de Azure AI Translator",
            "Spanish",
            "Italian",
        ],
        [
            "Bonjour à tous.",
            "French",
            "Arabic",
        ],
    ],
)

translator_webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://35b86eac92f9cf045f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
